In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL"
os.chdir(path)
os.listdir(path)
import os
print(os.getcwd())

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL


In [2]:
!pip install accelerate
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import pandas as pd

data_path = "AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt"

try:
    # 尝试使用 ISO-8859-1 编码读取数据
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1')
except UnicodeDecodeError:
    # 如果 ISO-8859-1 编码失败，则忽略错误的字符
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1', errors='ignore')

print(data.head())
print(data.info())

  Short_form Long_form Abbreviation  Start_pos  End_pos  \
0         AB  abortion          AB.      231.0    233.0   
1         AB  abortion          AB.      249.0    251.0   
2         AB  abortion           AB      223.0    224.0   
3         AB  abortion          AB.      194.0    196.0   
4         AB  abortion           AB      114.0    115.0   

                     Section_info  \
0                             NaN   
1                             NaN   
2                 PAST OB HISTORY   
3  HISTORY OF THE PRESENT ILLNESS   
4             PAST OB-GYN HISTORY   

                                             Context  
0  _%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...  
1  She is now bleeding quite heavily. Ultrasound ...  
2  ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...  
3  She had a pelvic ultrasound at Park Nicollet o...  
4  On _%#MMDD2007#%_, normal anatomy with anterio...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (to

In [4]:
# 假设 data 是 pandas DataFrame
labels = data['Long_form'].tolist()  # 从 DataFrame 中提取标签列
texts = data['Context'].tolist()  # 从 DataFrame 中提取文本列

# 创建从类别名称到整数索引的映射
label_list = sorted(set(labels))  # 创建一个包含所有唯一标签的列表
label_map = {label: idx for idx, label in enumerate(label_list)}

# 创建从整数索引到类别名称的逆映射
inverse_label_map = {idx: label for label, idx in label_map.items()}

# 打印标签映射以确保其正确性
print("Label Map:", label_map)
print("Inverse Label Map:", inverse_label_map)




print("Total labels:", len(label_map))
print("Total Inverse Label:", len(inverse_label_map))



# 将字符串标签转换为整数索引
labels_indexed = [label_map[label] for label in labels]

# 使用 train_test_split 分割数据集
from sklearn.model_selection import train_test_split
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels_indexed, test_size=0.2, random_state=42)

Label Map: {'(PO Box) C4': 0, '(class) IA': 1, '(complement) component 3': 2, '(complement) component 4': 3, '(cycle) IB': 4, '(device) MP': 5, '(device) PD': 6, '(diltiazem) DC': 7, '(drug) AC': 8, '(drug) DC': 9, '(drug) DT': 10, '(drug) ES': 11, '(drug) IT': 12, '(drug) MP': 13, '(drug) MS': 14, '(drug) NP': 15, '(drug) OR': 16, '(drug) PAC': 17, '(drug) PD': 18, '(drug) PR': 19, '(drug) RT': 20, '(grade) IA': 21, '(grade) IB': 22, '(stage) C3': 23, '(stage) IA': 24, '(stage) IB': 25, '(status) IA': 26, '(status) IB': 27, '(type) IA': 28, '(type) IB': 29, 'American Society of Anesthesiologists': 30, 'American Society of Anesthesiologists:ASA': 31, 'BK (virus)': 32, 'C-reactive': 33, 'California': 34, "Children's Depression Inventory": 35, 'District of Columbia': 36, 'Fairview Southdale Hospital': 37, 'GENERAL ENGLISH': 38, 'Iowa': 39, 'Los Angeles': 40, 'Louisiana': 41, 'MISTAKE:EZ PAP': 42, 'MISTAKE:Oncotype DX': 43, 'MISTAKE:abduction': 44, 'Mall of America:MOA': 45, 'NAME': 46, '

In [5]:


# 显示一些基本信息以确保数据正确处理
print(f"Number of training samples: {len(texts_train)}")
print(f"Number of test samples: {len(texts_test)}")
print("Example of processed data:")
print(texts_train[0], labels_train[0])


Number of training samples: 30000
Number of test samples: 7500
Example of processed data:
Will get his home medications and continue those. Will ask neurology to review his antiepileptic regimen and have spoken with Dr. _%#NAME#%_. We will have OT and PT evaluate the patient. Suspect that he may need a TCU versus home with additional services. LABORATORY DATA: That was performed here includes the following laboratory tests. 267


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch

class ClinicalTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 加载预训练的Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# 在将数据传给tokenizer之前，确保所有文本都是字符串
texts_train = [str(text) for text in texts_train]
texts_test = [str(text) for text in texts_test]

# 现在尝试编码
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=512)


# 编码数据
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=512)

# 创建数据集
train_dataset = ClinicalTextDataset(train_encodings, labels_train)
test_dataset = ClinicalTextDataset(test_encodings, labels_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 加载预训练的BERT模型，添加一个顶层用于分类
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map)).to(device)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 训练模型
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,5.871100
200,5.572700
300,5.200700
400,4.988800
500,4.775100
600,4.452200
700,3.970000
800,3.502900
900,2.940400
1000,2.441000


TrainOutput(global_step=11250, training_loss=0.7741411438835992, metrics={'train_runtime': 2016.3091, 'train_samples_per_second': 44.636, 'train_steps_per_second': 5.58, 'total_flos': 2.375419696128e+16, 'train_loss': 0.7741411438835992, 'epoch': 3.0})

In [21]:
# 计算训练集上的准确率
train_result = trainer.evaluate(train_dataset)
print("Training set accuracy:", train_result['eval_accuracy'])

KeyboardInterrupt: 

In [7]:
# 保存训练好的模型
model_path = "./biobert_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./biobert_finetuned/tokenizer_config.json',
 './biobert_finetuned/special_tokens_map.json',
 './biobert_finetuned/vocab.txt',
 './biobert_finetuned/added_tokens.json')

In [8]:
# 确保模型的分类头设置正确
print("Expected number of labels (from label map):", len(label_map))
model_config = model.config  # 假设模型有一个可访问的配置属性
print("Number of labels in model configuration:", model_config.num_labels)




Expected number of labels (from label map): 351
Number of labels in model configuration: 351


In [16]:
from transformers import BertTokenizer, Trainer, TrainingArguments

# 假设你已经有了 num_labels 和 device 的定义
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

model_path = "./biobert_finetuned"

# 加载预训练并微调过的模型
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(label_map)).to(device)

# 加载与模型相对应的分词器
tokenizer = BertTokenizer.from_pretrained(model_path)



# 使用数据加载器
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# 初始化用于跟踪正确预测和总预测的变量
correct_predictions = 0
total_predictions = 0

# 测试训练过的模型
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    pred_indices = outputs.logits.argmax(-1).tolist()  # 获取预测的类别索引
    actual_indices = batch['labels'].tolist()  # 获取实际的类别索引

    # 累计正确预测的数量
    correct_predictions += sum(pred == actual for pred, actual in zip(pred_indices, actual_indices))
    total_predictions += len(batch['labels'])  # 更新总预测数量

# 计算准确率
accuracy = correct_predictions / total_predictions * 100  # 计算百分比
print(f"Accuracy: {accuracy:.2f}%")






Accuracy: 94.21%


In [20]:
# 引入需要的库
import torch
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载未训练的BERT模型，指定类别数
model_name = 'bert-base-uncased'
untrained_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))
untrained_model.to(device)

# 创建数据加载器
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 初始化统计变量
correct_predictions = 0
total_predictions = 0

# 遍历测试数据
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = untrained_model(**batch)
    pred_indices = outputs.logits.argmax(-1).tolist()

    # 比较预测结果与实际标签
    actual_indices = batch['labels'].tolist()
    correct_predictions += sum(p == a for p, a in zip(pred_indices, actual_indices))
    total_predictions += len(batch['labels'])

# 计算并输出准确率
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy of untrained model: {accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy of untrained model: 0.19%


In [25]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
!pip install datasets
from datasets import load_metric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [26]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer

# 路径，这里假设你的模型和分词器都保存在同一路径下
model_path = "./biobert_finetuned"
model = BertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = BertTokenizer.from_pretrained(model_path)

metric = load_metric("accuracy")

# 定义计算指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',            # 输出目录
    per_device_eval_batch_size=16,     # 每个设备的评估批次大小
    do_train=False,                    # 不进行训练
    do_eval=True,                      # 进行评估
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,        # 指定训练集为训练数据
    eval_dataset=train_dataset,         # 指定评估集为训练数据
    compute_metrics=compute_metrics     # 添加计算准确率的函数
)
# 计算训练集上的准确率
train_eval_result = trainer.evaluate()
print("Training Set Accuracy:", train_eval_result.get('eval_accuracy'))


<ipython-input-26-154874490d67>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Training Set Accuracy: 0.9701333333333333
